In [2]:
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm=ChatOpenAI(temperature=0.1)


class CalculatorToolArgsSchema(BaseModel):
    a: int = Field(description="The first number")
    b: int = Field(description="The second number")

class CalculatorTool(BaseTool):
    name = "CalculatorTool"
    description = """
    Use this to perform sums of two numbers.
    Thr first and second arguments should be numbers.
    Only receives two arguments.
    """
    args_schema : Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b):
        return a + b


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools = [
        CalculatorTool(),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

agent.invoke(prompt)




> Entering new AgentExecutor chain...

Invoking: `CalculatorTool` with `{'a': 35539, 'b': 92487}`


128026
Invoking: `CalculatorTool` with `{'a': 128026, 'b': 72120}`


200146
Invoking: `CalculatorTool` with `{'a': 200146, 'b': 194029}`


394175
Invoking: `CalculatorTool` with `{'a': 394175, 'b': 57363}`


451538
Invoking: `CalculatorTool` with `{'a': 451538, 'b': 6572}`


458110
Invoking: `CalculatorTool` with `{'a': 458110, 'b': 3500}`


461610
Invoking: `CalculatorTool` with `{'a': 461610, 'b': 55200}`


516810
Invoking: `CalculatorTool` with `{'a': 516810, 'b': 7616}`


524426
Invoking: `CalculatorTool` with `{'a': 524426, 'b': 2912}`


527338The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.

> Finished chain.


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}